In [2]:
import numpy as np
#import pandas as pd
#import bz2file as bz2
import os
from typing import Tuple, Optional


# Introduction to the Gisette Dataset

The Gisette dataset is a well-known benchmark dataset in the field of machine learning, particularly used for feature selection and binary classification tasks. It was originally part of the NIPS 2003 feature selection challenge. The dataset consists of handwritten digit images, where the task is to distinguish between the digits '4' and '9'.

## Dataset Characteristics

- **Features:** The dataset contains 5000 features, many of which are redundant or irrelevant, making it a good test for feature selection algorithms.
- **Instances:** There are 7000 instances in the training set and 1000 instances in the test set.
- **Classes:** The labels are binary, with two classes representing the digits '4' and '9'.

## Usage

The Gisette dataset is often used to evaluate the performance of various machine learning algorithms, especially those designed for high-dimensional data. It provides a challenging testbed for algorithms due to its high dimensionality and the presence of irrelevant features.

## References

put a reference here

All features are scaled.

The Gisette dataset is often used to evaluate the performance of various machine learning algorithms, especially those designed for high-dimensional data. It provides a challenging testbed for algorithms due to its high dimensionality and the presence of irrelevant features.
each line is like this: 
-1 1:-1 2:-1 3:0.913914 4:-1 5:-1 6:0.4530 ...
the first number is either 1 or -1 (label y)
 and it is followed by 5000 pairs of the form integer_index. the floats are the x values


In [3]:
from src.utils import read_gisette_data

In [54]:
MAX_LINES = 4500
file_path_train = os.path.join("..","data","gisette_scale.bz2")
file_path_test = os.path.join("..","data","gisette_scale.t.bz2")


y_train, X_train = read_gisette_data(file_path_train, max_lines=MAX_LINES)
y_test, X_test = read_gisette_data(file_path_test, max_lines=MAX_LINES)

## SVM problem definition

* the optimization problem we should solve it the following one:
$$
\begin{equation}
\begin{aligned}
& \min \quad \frac{1}{2} \|\mathbf{w}\|^2 + C \sum_{i=1}^m \xi_i \\
& \text{subject to} \quad y_i (\mathbf{w} \cdot \mathbf{x}_i + b) \geq 1 - \xi_i, \quad \xi_i \geq 0, \quad i = 1, \dots, m
\end{aligned}
\end{equation}
$$

According to Platt's algorithm [put reference here] it is preferrable to solve the dual, which is the following:

$$
\begin{equation}
\begin{aligned}
& \max_{\boldsymbol{\alpha}} \quad \sum_{i=1}^n \alpha_i - \frac{1}{2} \sum_{i=1}^n \sum_{j=1}^n \alpha_i \alpha_j y_i y_j \mathbf{x}_i \cdot \mathbf{x}_j \\
& \text{subject to} \sum_{i=1}^n \alpha_i y_i = 0 \\
& \quad \quad \quad \quad 0 \leq \alpha_i \leq C, \quad i = 1, \dots, m
\end{aligned}
\end{equation}
$$
     

In [55]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, f1_score, precision_score

# Create an SVM classifier
svm = SVC(kernel='linear', C=100)

# Train the SVM classifier
svm.fit(X_train, y_train)


SVC(C=100, kernel='linear')

In [56]:
# Predict on the test data
y_pred = svm.predict(X_test)

# Calculate the accuracy
accuracy = accuracy_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)

print(f'Accuracy of SVM on test set: {accuracy:.2f}')
print(f'F1 of SVM on test set: {f1:.2f}')
print(f'Precision of SVM on test set: {precision:.2f}')



Accuracy of SVM on test set: 0.97
F1 of SVM on test set: 0.97
Precision of SVM on test set: 0.97


# Custom SMO algorithm

* Create the SMO algorith
* use classes in scikit-learn similar manner, so that has fit and predict methods for training and inference
* the cor of the classifier class are the two functions described in Platt's paper `take_step` and `examine_example`
* The main routine has been replaced with `fit` and has the two 

In [86]:
class SVM_classifier:
    def __init__(self, X, y, kernel:str ='linear', C:float =1, epsilon:float = 1e-8, tol:float = 0.001, max_iter:int= 500):
        self.X = X
        self.y = y
        self.kernel = kernel
        self.kernel_func = self.select_kernel(self.kernel)
        self.C = C
        self.epsilon = epsilon # error margin 
        self.tol = tol # tolerance for KKT
        self.max_iter = max_iter
        self.m, self.n = np.shape(self.X) # m is number of samples, n number of features
        
        self.alphas = np.zeros(self.m)
        self.Error_cache = - y 
        
        # If the kernel is linear we can store a single weight vector and use the alternative implemented in SVM
        
        self.w = np.zeros(self.n)
        self.b = 0 # intercept            
        
        
    def select_kernel(self, kernel:str):
        
        ''' We have to choose a kernel based on the kernel type argument
        here we can use only linear or the gaussion, no other kernels are available'''
        
        if kernel == 'linear':
            return self.linear_kernel
        elif kernel == 'rbf':
            return self.rbf_kernel
        else:
            raise ValueError(f"Unsupported kernel type: {kernel}")
    
    def linear_kernel(self, x1: np.ndarray, x2: np.ndarray) -> np.ndarray:
        return x1 @ x2.T
    
    def rbf_kernel(self, x1: np.ndarray, x2: np.ndarray) -> np.ndarray:
        if x1.ndim == 1:
            x1 = x1.reshape(1, -1)
        if x2.ndim == 1:
            x2 = x2.reshape(1, -1)

        # Compute the squared Euclidean distance between each pair of points
        squares = np.sum(x1*x1, axis=1).reshape(-1, 1) + np.sum(x2*x2, axis=1) - 2 * np.dot(x1, x2.T)

        # Compute the Gaussian kernel with auto-scaling
        gamma = 1.0/x1.size
        K = np.exp(-gamma*squares)
        return K  
    
    
    def predict(self,x: np.ndarray) -> Tuple[np.ndarray, np.ndarray]:
        """ Predicts the labels for the instance
            and the respective score."""

        # if self.kernel != 'linear':
        #     w = self.alphas * self.y
        #     scores = w @ self.kernel_func(self.X, x) + self.b
        # else:
        #     scores = self.w @ self.kernel_func(self.X,x) + self.b
        
        scores = (self.alphas * self.y) @ self.kernel_func(self.X,x) - self.b
        pred = np.sign(scores)

        return pred, scores
    
    def take_step(self, i1:int=None, i2:int=None) -> int:
        """
        takes one step of the SMO algorithm
        :param i1: i1-th training instance 
        :param i2: i2-th training instance
        :return: 1 if success else 0
        """
        #print("I an in take_step.")
        if i1==i2:
            #print("i1==i2\n0 returned.")
            return 0
        
        # Set all required parameters
        a1 = self.alphas[i1]
        a2 = self.alphas[i2]
        
        x1 = self.X[i1,:]
        x2 = self.X[i2,:]
        
        y1 = self.y[i1]
        y2 = self.y[i2]
        
        E1 = self.Error_cache[i1] # HERE is the error
        E2 = self.Error_cache[i2]
        
        # Define parameter s
        s = y1*y2
        
        # Compute L, H via equations (13) and (14) from Platt
        if y1!=y2:
            L = max(0,a2-a1)
            H = min(self.C,self.C+a2-a1)
        else:
            L = max(0,a2+a1-self.C)
            H = min(self.C,a2+a1)
            
        if L==H:
            #print("L==H\n0 returned.")
            return 0
        
        k11 = self.kernel_func(x1,x1)
        k22 = self.kernel_func(x2,x2)
        k12 = self.kernel_func(x1,x2)
        
        # Compute the second derivative of the objective function along the diagonal line
        eta = k11 + k22 - 2.0*k12
        
        if eta > 0:
            # Normal circumstances, using Equations (16)-(18) to compute a1 and a2
            a2_new = a2 +y2*(E1-E2)/eta
            
            if a2_new>=H:
                a2_new = H
            if a2_new<=L:
                a2_new = L
        else:
            # Strange case, we use Equations (19)
            f1 = y1*(E1 + self.b) - a1*k11 - s*a2*k12
            f2 = y2*(E2 + self.b) - s*a1*k12 - a2*k22
            L1 = a1 + s*(a2 - L)
            H1 = a1 + s*(a2 - H)
            psi_L = L1*f1 + L*f2 + 0.5*L1*L1*k11 + 0.5*L*L*k22 + s*L*L1*k12
            psi_H = H1*f1 + H*f2 + 0.5*H1*H1*k11 + 0.5*H*H*k22 + s*H*H1*k12
            
            if psi_L < (psi_H - self.epsilon):
                a2_new = L
            elif psi_L > (psi_H + self.epsilon):
                a2_new = H
            else:
                a2_new = a2

        # if a2 very close to zero or C set a to 0 or C respectively
        #if a2 < (10 ** (-8)):
        #    a2 = 0.0
        #elif a2 > self.C - (10**-8):
        #    a2 = self.C
        
        if np.abs(a2_new - a2) < (self.epsilon * (a2_new + a2 + self.epsilon)):
            #print("off numerical tolerance\n0 returned.")
            return 0
        
        # Calculate a1_new
        a1_new = a1 + s*(a2 - a2_new)
        
        # Push alphas to boundaries
        if a1_new < self.epsilon:
            a1_new = 0
        if a1_new > (self.C - self.epsilon):
            a1_new = self.C
        
        # Update threshold b
        b1 = self.b + E1 + y1*(a1_new - a1)*k11 + y2*(a2_new - a2)*k12
        b2 = self.b + E2 + y1*(a1_new - a1)*k12 + y2*(a2_new - a2)*k22
        
        if 0 < a1_new < self.C:
            b_new = b1
        elif 0 < a2_new < self.C:
            b_new = b2
        else:
            b_new = 0.5*(b1 + b2)
            
        # Update weight's vector if Linear kernel
        if self.kernel == 'linear':
            self.w = self.w + y1*(a1_new - a1)*x1 + y2*(a2_new - a2)*x2
            
        # Update Error_cache using alphas (see reference)
        
        # if a1 & a2 are not at bounds, the error will be 0
        self.Error_cache[i1] = 0
        self.Error_cache[i2] = 0
        
        # Update error for non boundary elements
        inner_indices = [idx for idx, a in enumerate(self.alphas) if 0 < a < self.C]
        for i in inner_indices:
            self.Error_cache[i] += \
                 y1*(a1_new - a1)*self.kernel_func(x1, self.X[i,:]) \
               + y2*(a2_new - a2)*self.kernel_func(x2, self.X[i,:]) \
               + (self.b - b_new)
        
        
        # Update alphas
        self.alphas[i1] = a1_new
        self.alphas[i2] = a2_new
        
        # Update b
        self.b = b_new
        
        #print("successfull pass")
        return 1 # sucessfull pass
    
    def examine_example(self, i2:int=None):
        """
        Examine the i2-th example in the algorithm to determine
        if eligible for usage in optimization pair
        :param i2: example to examine
        :return: 1 if successful, 0 if not
        """
        
        y2 = self.y[i2]
        a2 = self.alphas[i2]
        E2 = self.Error_cache[i2]
        r2 = E2 * y2
        #print("I am in examine_example()")
        # Check if error is within tolerance
        if (r2 < -self.tol and a2 < self.C) or (r2 > self.tol and a2 > 0):
            #print("Error within tolerance.")
            # If there are more than one non-bound elements use the second heuristic 
            if np.count_nonzero((0 < self.alphas) & (self.alphas < self.C)) > 1:
                
                # use section 2.2 to select i1
                if E2 > 0:
                    i1 = np.argmin(self.Error_cache)
                else:
                    i1 = np.argmax(self.Error_cache)

                if self.take_step(i1, i2):
                    return 1
                
            # Loop over all non-zero and non-C alpha, starting at a random point

            # Get indices where 0 < alpha < self.C
            i1_array = np.where((0 < self.alphas) & (self.alphas < self.C))[0]

            # Roll the array by a random number of positions to ensure that we will pass all
            random_shift = np.random.choice(np.arange(self.m))
            i1_list = np.roll(i1_array, random_shift)
            
            # Loop over all non-boundary elements
            for i1 in i1_list:
                if self.take_step(i1, i2):
                    return 1

            # Loop over all possible alpha elements, starting at a random point
            i1_list = np.roll(np.arange(self.m), np.random.choice(np.arange(self.m)))
            for i1 in i1_list:
                if self.take_step(i1, i2):
                    return 1
                
                    
        return 0
    
    def fit(self) -> None:
        """This is the equivalent of the main routine in the original SMO paper.
            We use it for training the algorithm."""
        iteration_number = 0 # We count the number of iterations and bounded below max_iter
        numbers_changed = 0
        examine_all = True
    
        while numbers_changed > 0 or examine_all:
            
            if iteration_number >= self.max_iter:
                break
                
            numbers_changed = 0
            if examine_all:
                # Loop i over all training examples
                for i in range(self.m):
                    numbers_changed += self.examine_example(i)
            
            else: 
                # Loop i over examples where alpha is not 0 & not C
                i_array = np.where((0 < self.alphas) & (self.alphas < self.C))[0]
                for i in i_array:
                    numbers_changed += self.examine_example(i)
                    
            if examine_all:
                examine_all = False
            if numbers_changed == 0:
                examine_all = True
                
            iteration_number += 1
            
            #print(f"Iteration:{iteration_number} ended.")
            #return self.b, self.w
        
        
        

In [92]:
model = SVM_classifier(X_train, y_train, kernel='linear', C=10)
model.fit()

Iteration:1 ended.
Iteration:2 ended.
Iteration:3 ended.
Iteration:4 ended.
Iteration:5 ended.
Iteration:6 ended.
Iteration:7 ended.
Iteration:8 ended.
Iteration:9 ended.
Iteration:10 ended.
Iteration:11 ended.
Iteration:12 ended.
Iteration:13 ended.
Iteration:14 ended.
Iteration:15 ended.
Iteration:16 ended.
Iteration:17 ended.
Iteration:18 ended.
Iteration:19 ended.
Iteration:20 ended.
Iteration:21 ended.
Iteration:22 ended.
Iteration:23 ended.
Iteration:24 ended.
Iteration:25 ended.
Iteration:26 ended.
Iteration:27 ended.
Iteration:28 ended.
Iteration:29 ended.
Iteration:30 ended.
Iteration:31 ended.
Iteration:32 ended.
Iteration:33 ended.
Iteration:34 ended.
Iteration:35 ended.
Iteration:36 ended.
Iteration:37 ended.
Iteration:38 ended.
Iteration:39 ended.
Iteration:40 ended.
Iteration:41 ended.
Iteration:42 ended.
Iteration:43 ended.
Iteration:44 ended.
Iteration:45 ended.
Iteration:46 ended.
Iteration:47 ended.
Iteration:48 ended.
Iteration:49 ended.
Iteration:50 ended.
Iteration

In [93]:
# Predict on the test data
y_pred, scores = model.predict(X_test)

# Calculate the accuracy
accuracy = accuracy_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)

print(f'Accuracy of SVM on test set: {accuracy:.2f}')
print(f'F1 of SVM on test set: {f1:.2f}')
print(f'Precision of SVM on test set: {precision:.2f}')

Accuracy of SVM on test set: 0.79
F1 of SVM on test set: 0.82
Precision of SVM on test set: 0.72


In [63]:
model.max_iter

500

In [64]:
model.alphas * model.y

array([ 0.01302314, -0.00079364,  0.00079364, ...,  0.00074602,
        0.00097505,  0.00081396])

In [35]:
model.Error_cache

array([ 1, -1,  1, ..., -1, -1, -1])

In [25]:
model.w @ model.kernel_func(model.X,X_test) + model.b

ValueError: matmul: Input operand 1 has a mismatch in its core dimension 0, with gufunc signature (n?,k),(k,m?)->(n?,m?) (size 6000 is different from 5000)

In [65]:
scores

array([ 154.60336729,  312.04830122,  145.00765438,  278.12430417,
        299.48779679,   71.26228702, -102.98908487,   98.01602573,
         41.54913322,   71.68895929,  254.6402256 ,  229.69593321,
        188.91696722,  284.84625698,   52.21427784,  249.39773982,
        299.09237365,  280.495324  ,  284.89015868,  314.441512  ,
         30.75190122,  315.71076547,  292.14330305,  244.86023937,
        227.01679604,   94.91369686,  298.25585023,  313.92748312,
        216.60662911,  233.44074888,  193.99826832,  253.700384  ,
        137.95949434,  272.28310868,  243.29085382,  307.21791766,
         87.6843977 ,  300.96957551,   49.00330707, -146.05886609,
        291.53471036,   60.45145777,  272.68651169,  295.94075274,
        -40.93146466, -138.76999412,  270.67131207,  169.05336082,
        175.41469554,  208.69377914,  245.63514145,   83.79987473,
         77.84801185,   34.77912159,  236.93496586,  311.91305272,
         68.04736882,  -12.36721478,  217.54270247,  282.33363

In [ ]:
(self.alphas * self.y) @ self.kernel_func(self.X,x) + self.b

In [67]:
model.alphas * model.y

array([ 0.01302314, -0.00079364,  0.00079364, ...,  0.00074602,
        0.00097505,  0.00081396])

In [68]:
model.kernel_func(X_train,X_test)

array([[2747.60037936, 2691.65263868, 2588.55710927, ..., 2905.65651177,
        2785.0318925 , 2839.62542968],
       [3471.33035727, 3077.35893934, 3195.77757655, ..., 2962.65789824,
        3569.67161785, 3405.76488456],
       [3626.59998421, 3286.26321253, 3295.53571808, ..., 3168.44028735,
        3428.5026344 , 3337.9038238 ],
       ...,
       [3545.43516493, 3360.39531991, 3229.80668907, ..., 3261.25575398,
        3523.05201928, 3404.87382227],
       [3598.23034808, 3604.4132433 , 3546.45090507, ..., 3423.837775  ,
        3615.09305959, 3517.72924292],
       [3464.79928594, 3554.76281547, 3212.17744376, ..., 3274.45945418,
        3512.76915348, 3406.64358457]])

In [79]:
 X_train @ X_test[50,:]

array([2636.3739335 , 3303.2276783 , 3446.44640771, ..., 3506.38525004,
       3802.78997489, 3521.13239304])

In [85]:
(model.alphas * model.y) @ model.kernel_func(X_train,X_test) - model.b

array([  40.19894287,  197.64387679,   30.60322996,  163.71987974,
        185.08337237,  -43.14213741, -217.39350929,  -16.38839869,
        -72.8552912 ,  -42.71546513,  140.23580118,  115.29150879,
         74.51254279,  170.44183256,  -62.19014658,  134.9933154 ,
        184.68794923,  166.09089958,  170.48573425,  200.03708758,
        -83.65252321,  201.30634105,  177.73887863,  130.45581495,
        112.61237161,  -19.49072756,  183.8514258 ,  199.5230587 ,
        102.20220469,  119.03632446,   79.59384389,  139.29595958,
         23.55506992,  157.87868426,  128.8864294 ,  192.81349324,
        -26.72002672,  186.56515109,  -65.40111735, -260.46329051,
        177.13028594,  -53.95296665,  158.28208726,  181.53632831,
       -155.33588909, -253.17441855,  156.26688765,   54.64893639,
         61.01027112,   94.28935471,  131.23071702,  -30.60454969,
        -36.55641257,  -79.62530284,  122.53054144,  197.50862829,
        -46.35705561, -126.7716392 ,  103.13827804,  167.92920

In [94]:
model.w

array([ 0.02077156,  0.00629244, -0.44885489, ..., -0.07390252,
       -0.00827287, -0.42565927])

In [100]:
model.w[0]

np.float64(0.02077155735911605)

In [101]:
# this is w for linear
(model.alphas * model.y) @ X_train

array([ 0.02077156,  0.00629244, -0.44885489, ..., -0.07390252,
       -0.00827287, -0.42565927])